In [3]:
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.optimize import curve_fit
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import folium
import ast


# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 100


In [16]:
df = pd.read_json('all_relay_data.json')


### Missing Data

Alright, we need to do something about the missing data in the dataframe. Currently we have replaced the NaN values in the dataset with an empty string. This is going to be fine for handling this data, but it is not going to be fine for the actual missing data fetched from the IP Stack API. So there are a few reasons IP Stack API is not able to fetch data here is some information about the [IP Stack possible error codes](https://ipstack.com/documentation). 

There seems to be one specific code that causes IP stack's API to not be able to retrieve any information about the IP address. This is the `{'code': 106, 'type': 'invalid_ip_address', 'info': 'The IP Address supplied is invalid.'}` error. So we need to handle this error. 

### Possible solutions:
1. Handle the error in the IP Stack API python script I wrote to deal with these errors.
2. Clean the dataframe by removing the rows that have the error code.



so what I ended up doing is just looking for all the connection that were nan (floats) and then checked if the `len of the strings is < 2` and if it is then I removed that row from the dataset. Then I needed to replace the single quotes with double quotes in the connections column to utilize the json.loads function. After that the last step was to make a json file.

In [34]:
# create a copy of the dataframe for testing purposes
testing = df.copy()

# This is code below will create a list of indexes of rows with missing data to be dropped
def drop_list(df):

	drop_list = []

	for i in range(len(df)):
		if df.connection[i] is None or len(df.connection[i]) < 2:
			drop_list.append(i)

	return drop_list



In [35]:
testing.drop(testing.index[drop_list(df)], inplace=True)
testing

,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail
0,54.220.20.40,3002.0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.353889,-6.243330,L,Leinster,None,None,NaN,None
1,relays.mainnet.stakenuts.com,3001.0,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json,199.247.23.219,Frankfurt am Main,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",DE,Germany,50.113701,8.711900,HE,Hesse,60314,None,NaN,None
2,relay.zenithpool.io,31400.0,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io,193.122.12.254,Seattle,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,47.611012,-122.333519,WA,Washington,98101,None,NaN,None
3,95.183.53.20,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.374168,8.536950,ZH,Zurich,8043,None,NaN,None
4,95.217.222.194,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.217.222.194,Tuusula,"{'asn': 24940, 'isp': 'Hetzner Online Gmbh'}",FI,Finland,60.426418,25.011320,18,Uusimaa,04300,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,188.166.12.123,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,188.166.12.123,Diemen,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",NL,Netherlands,52.309052,4.940190,NH,North Holland,1101,None,NaN,None
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,46.101.17.79,Blackheath,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",GB,United Kingdom,51.512218,NaN,ENG,England,EC2R,None,NaN,None
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,None,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,None,None,None,https://git.io/J6rex,72.86.45.91,Ashburn,"{'asn': 701, 'isp': 'Mci Communications Servic...",US,United States,39.039570,-77.481613,VA,Virginia,20147,None,NaN,None
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,None,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba6499

In [21]:
len(df.connection[0])

2

In [36]:
# Need to replace the single quotes with double quotes in the dataframe column connection 
# in order to be able to use the json.loads function later on

# testing.connection = testing.connection.replace({'\'': '"'}, regex=True)




In [37]:
# Read in the data from the json file using the json.loads function
# This will create a string with the correct format for which we can make into a dataframe

# data = json.load(open('relay_data_clean.json'))
# data = pd.DataFrame(data)
# data.connection[0]


In [38]:
# This is the code to create a list of dict from the list of json string

# def json_to_lst_dicts(data):
# 	lst = []
# 	for i in range(len(data)):
# 		if len(data['connection'][i]) > 0:
# 			result = json.loads(data['connection'][i])
# 			lst.append(result)
# 	return lst

# testing.connection = json_to_lst_dicts(data)

In [39]:
# Let's make sure it all worked as planned that when we attempt to check the connection column's
# entries type it is a dict

type(testing.connection[0])

dict

In [53]:
# Now we can easily grab (key, value) pairs from the connection column
testing.connection[0].get('isp')

'amazon.com, Inc'

In [54]:
testing 

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,metadata_hash,pool_id_hex,homepage,name,pool_id,ticker,url,ip host,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail
0,NaN,NaN,54.220.20.40,NaN,3002.0,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.353889,-6.243330,L,Leinster,NaN,NaN,NaN,NaN
1,relays.mainnet.stakenuts.com,NaN,NaN,NaN,3001.0,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json,198.13.62.211,Kawasaki,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",JP,Japan,35.583511,139.746155,14,Kanagawa,143-0006,NaN,NaN,NaN
2,relay.zenithpool.io,NaN,NaN,NaN,31400.0,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io,129.146.210.203,Phoenix,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,33.509380,-112.082550,AZ,Arizona,85013,NaN,NaN,NaN
3,NaN,NaN,95.183.53.20,NaN,6060.0,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.374168,8.536950,ZH,Zurich,8043,NaN,NaN,NaN
4,NaN,NaN,95.217.222.194,NaN,6060.0,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata,95.217.222.194,Tuusula,"{'asn': 24940, 'isp': 'Hetzner Online Gmbh'}",FI,Finland,60.426418,25.011320,18,Uusimaa,04300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,NaN,NaN,137.184.178.161,NaN,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b,23.147.229.150,Ann Arbor,{},US,United States,42.285641,-83.716728,MI,Michigan,48109,NaN,NaN,NaN
5414,NaN,NaN,46.101.17.79,NaN,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b,104.21.75.11,San Jose,"{'asn': 13335, 'isp': 'Cloudflare'}",US,United States,37.330528,-121.838226,CA,California,95122,NaN,NaN,NaN
5415,NaN,NaN,72.86.45.91,NaN,3001,18295901333,1,NaN,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,NaN,NaN,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,NaN,https://git.io/J6rex,64.225.79.144,Atlanta,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",US,United States,33.798458,-84.388283,GA,Georgia,30309,NaN,NaN,NaN
5416,r1.jaguares.org,NaN,NaN,NaN,3001,10008229492,1,NaN,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f

We need to now make a new isp and asn column for our df...:)

In [47]:
# create the isp and asn columns by using the get function with
testing['isp'] = testing.connection.apply(lambda x: x.get('isp', np.nan))
testing['asn'] = testing.connection.apply(lambda x: x.get('asn', np.nan))
testing[testing['ticker'] == 'PIADA']

,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail,isp,asn
3449,east.piada.io,3000,pool1hrv8gtrm0dgjg6zyss5uwa4nkruzfnh5vrdkr2say...,207194064280,67,PIADA provides educational content specializin...,b89a6afcc92714b580d5449b7691e81b3b8ac417cc664e...,b8d8742c7b7b512468448429c776b3b0f824cef460db61...,https://tinyurl.com/3n6ftstn,𝛑 + ₳ = ∞,PIADA,https://piada.io/piada.metadata.json,100.7.78.240,Bon Air,"{'asn': 701, 'isp': 'Mci Communications Servic...",US,United States,37.534168,-77.635902,VA,Virginia,23113,None,NaN,None,Mci Communications Services Inc. D/B/a Verizon...,701
3450,east.piada.io,3002,pool1hrv8gtrm0dgjg6zyss5uwa4nkruzfnh5vrdkr2say...,207194064280,67,PIADA provides educational content specializin...,b89a6afcc92714b580d5449b7691e81b3b8ac417cc664e...,b8d8742c7b7b512468448429c776b3b0f824cef460db61...,https://tinyurl.com/3n6ftstn,𝛑 + ₳ = ∞,PIADA,https://piada.io/piada.metadata.json,100.7.78.240,Bon Air,"{'asn': 701, 'isp': 'Mci Communications Servic...",US,United States,37.534168,-77.635902,VA,Virginia,23113,None,NaN,None,Mci Communications Services Inc. D/B/a Verizon...,701


In [49]:
# Save this to a json file called "cleaned"
testing.to_json('relay_data_clean.json')

In [56]:
# create a df with num relays per ISP and graph it
num_relays_per_isp = testing.isp.value_counts()
top_100_isp = num_relays_per_isp.head(100)
top_100_isp.name = 'count'

In [12]:
top_100_isp = top_100_isp.reset_index().replace({'index': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('index', sort=False).sum().sort_values(by='count', ascending=False)
# top_100_isp.rename(columns={'ISP':'Number of Relays'}, inplace=True)


In [46]:
# fig = px.bar(top_100_isp, x=top_100_isp.index, y=top_100_isp['count'], color=top_100_isp.index, width=1200, height=800, title='Top 100 ISPs',
#              labels={'index': 'ISP', 'count': 'Number of Relays'})

# fig.update_yaxes(automargin=True)

In [41]:
# # Geo Scatter Plot of the relays
# testing.fillna('', inplace=True)
# fig = px.scatter_geo(testing,
#                      lat = 'latitude',
#                      lon = 'longitude',
#                 #      locationmode='country names',
#                 #      locations="city",
#                      color="pool_id_hex", # which column to use to set the color of markers
#                      hover_name="ticker", # column added to hover information
#                      hover_data = ['live_stake', 'city', 'connection', 'delegators'],
#                 #      size="delegators", # size of markers
#                      projection="natural earth")
# fig.show()

In [15]:

# fig = px.scatter(top_100_isp, x=top_100_isp.index, y=top_100_isp.values, color=top_100_isp.index, width=1200, height=800, title='Top 100 ISPs', size=top_100_isp.values, hover_name=top_100_isp.index,
#                 labels={'index':'ISP', 'y':'Number of Relays'},)
             

# fig.update_yaxes(automargin=True)



In [16]:
# fig = px.bar(top_100_isp, x=top_100_isp.index, y=top_100_isp.values, color=top_100_isp.index, width=1200, height=800, title='Top 100 ISPs',
#              labels={'index': 'ISP', 'y': 'Number of Relays'})

# fig.update_yaxes(automargin=True)

In [17]:
# live_stake_per_isp = testing.groupby('isp').live_stake.sum()/1000000
# live_stake_per_isp

In [18]:
# rslt_df = df_stake_isp[df_stake_isp['live_stake'] > 100000000]
# rslt_df.sum()


In [19]:
# df_stake_isp = live_stake_per_isp.sort_values(ascending=False).to_frame()

# fig = px.bar(rslt_df, x=rslt_df.index, y=rslt_df.live_stake.values, color=rslt_df.index, width=1500, height=800, title='Top ISPs with a controlled Stake > 1 Billion ADA',
# 	    labels={'y': 'ADA Staked'})

# fig.update_yaxes(automargin=True)


In [20]:
# cardano_total_stake = testing.live_stake.sum()/1000000
# print("Total Stake controlled in ADA: "+str(cardano_total_stake))

# percentage_of_total_stake = live_stake_per_isp/cardano_total_stake * 100
# percentage_of_total_stake.sort_values(ascending=False)

In [21]:
# percentage_of_total_stake_df = percentage_of_total_stake.to_frame()
# percentage_of_total_stake_df.sort_values(by='live_stake', ascending=False, inplace=True)
# top_pools = percentage_of_total_stake_df.where(percentage_of_total_stake_df.live_stake > 1).dropna()
# top_pools

In [22]:
# fig = px.pie(percentage_of_total_stake_df, values=percentage_of_total_stake.values, names=percentage_of_total_stake.index, title='Percentage of Total Stake controlled in ADA Per ISP',)

# fig.update_traces(textposition='inside', textinfo='percent+label')

In [23]:
# fig = px.pie(top_pools, values=top_pools.live_stake.values, names=top_pools.index, title='Breakdown of the Top ',)

# fig.update_traces(textposition='inside', textinfo='percent+label')

In [24]:
# live_stake_per_isp.replace('amazon.com\s\Inc.','amazon.com\s\Inc',regex=True, inplace = True)



## Notes on what to do explore with the Data:

- Would be nice to get accurate picture of how much ada is stake per isp. Right now the issue is running the `pd.DataFrame.column.sum()` function is the dataframe has relays with same live stake being added up. For example 1 pool has 3 realys and 1 isp, but do to the way the dataset is built the 3 relays have a live stake attached so the 1 pool's live stake is essentially 3 times the actual value of the pool's stake.

- It would also be nice to know how many relays and individual pools per isp.

> ### Okay in this part let's attempt to get the right amount of ada staked in pools out of our dataframe which at the time of writing this it is about `23.5B` ada staked in pools. The approach I am going to take is going to be very simple. The goal is to make a new column we will call `ada_staked_per_pool`. In order to get this column we will just iterate through the rows and check if the previous row has the same pool_id or not, if it does not we will add the current rows live_stake value to  the new `ada_staked_per_pool` column, otherwise is the pool_id's are equal we will just add the current index to a drop_list to be dropped from the df.

Steps: 

1. create a new df called `df_ada_staked_per_pool` from the copy of the original df.

2. create empty dataframe called tmp_df

2. create a new empty list called drop_indexes

3. Create a for loop or a lambda function with map or something to iterate through the rows of the df and check the rows

In [25]:
df_ada_staked_per_pool = testing.copy()


In [26]:
df_ada_staked_per_pool

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,metadata_hash,pool_id_hex,homepage,name,pool_id,ticker,url,ip host,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail,isp,asn
0,,,54.220.20.40,,3002.0,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.3539,-6.24333,L,Leinster,,,,,"amazon.com, Inc",16509
1,relays.mainnet.stakenuts.com,,,,3001.0,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json,198.13.62.211,Kawasaki,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",JP,Japan,35.5835,139.746,14,Kanagawa,143-0006,,,,The Constant Company LLC,20473
2,relay.zenithpool.io,,,,31400.0,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io,129.146.210.203,Phoenix,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,33.5094,-112.083,AZ,Arizona,85013,,,,Oracle Corporation,31898
3,,,95.183.53.20,,6060.0,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.3742,8.53695,ZH,Zurich,8043,,,,Solar Communications Gmbh,197988
4,,,95.217.222.194,,6060.0,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata,95.217.222.194,Tuusula,"{'asn': 24940, 'isp': 'Hetzner Online Gmbh'}",FI,Finland,60.4264,25.0113,18,Uusimaa,04300,,,,Hetzner Online Gmbh,24940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,,,137.184.178.161,,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b,23.147.229.150,Ann Arbor,{},US,United States,42.2856,-83.7167,MI,Michigan,48109,,,,,
5414,,,46.101.17.79,,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b,104.21.75.11,San Jose,"{'asn': 13335, 'isp': 'Cloudflare'}",US,United States,37.3305,-121.838,CA,California,95122,,,,Cloudflare,13335
5415,,,72.86.45.91,,3001,18295901333,1,,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,,,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,,https://git.io/J6rex,64.225.79.144,Atlanta,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",US,United States,33.7985,-84.3883,GA,Georgia,30309,,,,Digitalocean LLC,14061
5416,r1.jaguares.org,,,,3001,10008229492,1,,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282f

In [27]:
df_ada_staked_per_pool = df_ada_staked_per_pool.drop_duplicates(subset=['pool_id'], keep='first')
df_ada_staked_per_pool


,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,metadata_hash,pool_id_hex,homepage,name,pool_id,ticker,url,ip host,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail,isp,asn
0,,,54.220.20.40,,3002.0,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.3539,-6.24333,L,Leinster,,,,,"amazon.com, Inc",16509
1,relays.mainnet.stakenuts.com,,,,3001.0,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json,198.13.62.211,Kawasaki,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",JP,Japan,35.5835,139.746,14,Kanagawa,143-0006,,,,The Constant Company LLC,20473
2,relay.zenithpool.io,,,,31400.0,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io,129.146.210.203,Phoenix,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,33.5094,-112.083,AZ,Arizona,85013,,,,Oracle Corporation,31898
3,,,95.183.53.20,,6060.0,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.3742,8.53695,ZH,Zurich,8043,,,,Solar Communications Gmbh,197988
5,,,139.162.148.181,,8088.0,4000924534,14,,79e7cf8d936bf0ced040516b288e2edc76f2f87af5400f...,6b6164af70861c5537cc9c8e50fdae35139ca2c8c6fbb4...,,,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...,,https://pool.adascan.net/meta/v1/poolmeta.json,139.162.148.181,Frankfurt am Main,"{'asn': 63949, 'isp': 'Linode LLC'}",DE,Germany,50.1109,8.6821,HE,Hesse,60311,,,,Linode LLC,63949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,relaynode1.thegroovynerd.com,,,,6000,144798771,1,Stake pool for the advancement of STEM (Scienc...,c7a6fc7a372ab238bd7e0206e4de521e216df68629e38a...,a9ec394d519fdc192707868c28f8285af37fe83c0c9ad4...,https://thegroovynerd.com,The Groovy Nerds Pool,pool148krjn23nlwpjfc8s6xz37pgttehl6pupjddg4q6e...,GNERD,https://git.io/JKPoj,146.158.31.5,Mariupol,"{'asn': 35273, 'isp': 'Pe Serhii Leonidovich P...",UA,Ukraine,47.0971,37.5434,14,Donetsk,87527,,,,Pe Serhii Leonidovich Ponomarov,35273
5411,bushido-relays.mainnet.samurai-pool.net,,,,3001,509616990,1,"Righteousness, Sincerity and Loyalty are virtu...",45097dc6108b8d65c0e14a94683dca5ff4913e1dcd77fd...,6688aae2724cb038fa348b3cacba7e002b653cc41dda86...,https://samurai-pool.net,Bushido Pool,pool1v6y24cnjfjcr37353v72ewn7qq4k20xyrhdgdd6sg...,BUSHI,https://www.samurai-pool.net/metadata.json,65.102.12.19,Portland,"{'asn': 209, 'isp': 'Centurylink Communication...",US,United States,45.4917,-122.774,OR,Oregon,97298,,,,Centurylink Communications LLC,209
5412,,,188.166.12.123,,6000,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,MIDN,https://git.io/JKN8b,45.92.126.202,Dinuba,"{'asn': 35913, 'isp': 'Dedipath'}",US,United States,36.4877,-119.503,C

In [28]:
pools_per_isp = df_ada_staked_per_pool.groupby('isp').pool_id.nunique().to_frame()
pools_per_isp.rename(columns={'pool_id': 'Number of Pools'}, inplace=True)

In [29]:
ada_per_isp = (df_ada_staked_per_pool.groupby('isp').live_stake.sum()/1000000).to_frame()
ada_per_isp

,live_stake
isp,
,2.877476e+08
1&1 Ionos Se,4.346519e+08
1&1 Versatel Deutschland Gmbh,7.491568e+06
31173 Services Ab,1.775868e+01
A1$1AG,5.327795e+06
...,...
nazwa.pl sp.z.o.o.,1.625940e+05
netplus.ch Sa,3.226819e+06
tele2 Sverige Ab,0.000000e+00


In [30]:
ada_per_isp = ada_per_isp.merge(pools_per_isp, left_index=True, right_index=True)

In [31]:
ada_per_isp

,live_stake,Number of Pools
isp,,
,2.877476e+08,50
1&1 Ionos Se,4.346519e+08,54
1&1 Versatel Deutschland Gmbh,7.491568e+06,2
31173 Services Ab,1.775868e+01,1
A1$1AG,5.327795e+06,4
...,...,...
nazwa.pl sp.z.o.o.,1.625940e+05,1
netplus.ch Sa,3.226819e+06,2
tele2 Sverige Ab,0.000000e+00,1


In [32]:
# pools_per_isp = df_ada_staked_per_pool.groupby(['isp','live_stake'])['pool_id'].count().to_frame()
# pools_per_isp = pools_per_isp.sort_values(by='pool_id',ascending=False)
ada_per_isp_100mil = ada_per_isp[ada_per_isp['live_stake'] > 100000000]
ada_per_isp_100mil.sort_values(by='live_stake', ascending=False, inplace=True)
testdf = ada_per_isp_100mil.copy()
testdf.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
ada_per_isp_100mil =  ada_per_isp_100mil.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
ada_per_isp_100mil.rename(index={'': 'Unknown'}, inplace=True)
ada_per_isp_100mil

,live_stake,Number of Pools
isp,,
"amazon.com, Inc",4.352034e+09,470
Hetzner Online Gmbh,2.153576e+09,246
Ovh Sas,1.982694e+09,133
Digitalocean LLC,1.852929e+09,408
Google LLC,1.560643e+09,214
Contabo Gmbh,1.242989e+09,231
Microsoft Corporation,8.710173e+08,96
The Constant Company LLC,7.553152e+08,81
Contabo Inc.,5.002857e+08,78


In [33]:
fig = px.bar(ada_per_isp_100mil, x=ada_per_isp_100mil.index, y=ada_per_isp_100mil.live_stake.values, color=ada_per_isp_100mil.index, width=1500, height=800, title='Top ISPs by number of pools and stake',
	    labels={'y': 'ADA Staked'}, hover_data=['Number of Pools'])

fig.update_layout(xaxis_tickangle=-90)


fig.update_yaxes(automargin=True)

In [34]:
df_69 = testing.copy()


In [35]:
occurrences = df_69.groupby(['isp']).size().to_frame()
occurrences.rename(columns={0:'Number of Relays'}, inplace=True)
occurrences.sort_values(by='Number of Relays', ascending=False, inplace=True)
occurrences

,Number of Relays
isp,
Digitalocean LLC,706
"amazon.com, Inc",653
Hetzner Online Gmbh,449
Contabo Gmbh,397
Google LLC,341
...,...
Siel Informacijske Resitve d.o.o.,1
Siminn Hf,1
Digi Spain Telecom s.l.u.,1


In [36]:
# df_69.sort_values(by='hex', ascending=False, inplace=True)
# df_69

In [37]:
occurrences.sort_values(by='Number of Relays', ascending=False, inplace=True)

In [38]:
occurrences.rename(index={'': 'Unknown'}, inplace=True)

In [39]:
top_isps = ada_per_isp_100mil.index.to_list()

In [40]:
occurrences.drop(top_isps, inplace=True)

In [41]:
occurrences.sort_values(by='Number of Relays', ascending=False, inplace=True)

In [42]:
occurrences

,Number of Relays
isp,
amazon.com Inc.,121
Akamai Technologies Inc.,51
Strato Ag,37
Charter Communications Inc,31
Sakura Internet Inc.,24
...,...
Delta Fiber Nederland B.V.,1
Siel Informacijske Resitve d.o.o.,1
Siminn Hf,1


# Using DNSpython to reolve hostnames to IP addresses


In [ ]:
import dns.resolver

result = dns.resolver.resolve('east.piada.io', 'A')

for val in result:
    print('A Record : ', val.to_text())

A Record :  100.7.78.240


In [45]:
data

,ip_host,port,pool_id,live_stake,delegators,description,hash,hex,homepage,name,ticker,url,resolved_ip,city,connection,country_code,country_name,latitude,longitude,region_code,region_name,zip,error,success,detail
0,54.220.20.40,3002.0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,49569650567271,3340,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,OCTAS,https://raw.githubusercontent.com/Octalus/card...,54.220.20.40,Dublin,"{'asn': 16509, 'isp': 'amazon.com, Inc'}",IE,Ireland,53.353889,-6.243330,L,Leinster,None,None,NaN,None
1,relays.mainnet.stakenuts.com,3001.0,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,19157905491154,165,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,NUTS,https://stakenuts.com/mainnet.json,199.247.23.219,Frankfurt am Main,"{'asn': 20473, 'isp': 'The Constant Company LLC'}",DE,Germany,50.113701,8.711900,HE,Hesse,60314,None,NaN,None
2,relay.zenithpool.io,31400.0,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,547697202732,9,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,ZEN,https://metadata.zenithpool.io,193.122.12.254,Seattle,"{'asn': 31898, 'isp': 'Oracle Corporation'}",US,United States,47.611012,-122.333519,WA,Washington,98101,None,NaN,None
3,95.183.53.20,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.183.53.20,Zürich,"{'asn': 197988, 'isp': 'Solar Communications G...",CH,Switzerland,47.374168,8.536950,ZH,Zurich,8043,None,NaN,None
4,95.217.222.194,6060.0,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,1862316910352,707,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,000,https://ispool.live/metadata,95.217.222.194,Tuusula,"{'asn': 24940, 'isp': 'Hetzner Online Gmbh'}",FI,Finland,60.426418,25.011320,18,Uusimaa,04300,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,137.184.178.161,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,137.184.178.161,Ashburn,{},US,United States,39.043701,-77.474197,VA,Virginia,20147,None,NaN,None
5505,46.101.17.79,6000,pool1pxsvz3yy606zr6qnhl2snvgzhcwny74pqaa2uvrk7...,130027614,1,"midnort staking pool, your nordic Cardano Stak...",cb738f574938eaa9e960e052f071bfb62e19950e1f3acf...,09a0c14484d3f421e813bfd509b102be1d327aa1077aae...,https://cryptoginger29.wixsite.com/midnort,midnort-ada,MIDN,https://git.io/JKN8b,46.101.17.79,Blackheath,"{'asn': 14061, 'isp': 'Digitalocean LLC'}",GB,United Kingdom,51.512218,NaN,ENG,England,EC2R,None,NaN,None
5506,72.86.45.91,3001,pool1k7gvq974d55yqcs4g6xyx4wy2fdlf97pffwgesjfq...,18295901333,1,None,92cf348da2c6a2a0666196009b4eb97101d3231955fd84...,b790c017d56d28406215468c4355c4525bf497c14a5c8c...,None,None,None,https://git.io/J6rex,72.86.45.91,Ashburn,"{'asn': 701, 'isp': 'Mci Communications Servic...",US,United States,39.039570,-77.481613,VA,Virginia,20147,None,NaN,None
5507,r1.jaguares.org,3001,pool1wz5klzzjstlzmqgv55q8zwaxfxvyjyjhq49u9hg3f...,10008229492,1,None,0915cb165e3532496e7183fce748827ac1f1aceebec819...,70a96f885282fe2d810ca500713ba64998491257054bc2...,None,None,None,https:

In [51]:
data = pd.read_json('relay_data_clean.json')
df_ada_staked_per_pool = data.copy()
df_ada_staked_per_pool = df_ada_staked_per_pool.drop_duplicates(subset=['pool_id'], keep='first')
pools_per_isp = df_ada_staked_per_pool.groupby('isp').pool_id.nunique().to_frame()
pools_per_isp.rename(columns={'pool_id': 'Number of Pools'}, inplace=True)
ada_per_isp = (df_ada_staked_per_pool.groupby('isp').live_stake.sum()/1000000).to_frame()
ada_per_isp = ada_per_isp.merge(pools_per_isp, left_index=True, right_index=True)
ada_per_isp_100mil = ada_per_isp[ada_per_isp['live_stake'] > 100000000]
ada_per_isp_100mil.sort_values(by='live_stake', ascending=False, inplace=True)
ada_per_isp_100mil =  ada_per_isp_100mil.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
ada_per_isp_100mil.rename(index={'': 'Unknown'}, inplace=True)
num_relays_per_isp = data.groupby(['isp']).size().to_frame()
num_relays_per_isp.rename(columns={0:'Number of Relays'}, inplace=True)
top_100_isp = num_relays_per_isp.head(100)
top_100_isp = top_100_isp.reset_index().replace({'isp': {'amazon.com Inc.':'amazon.com, Inc'}}).groupby('isp', sort=False).sum()
top_100_isp.sort_values(by='Number of Relays', ascending=False, inplace=True)

top_100_isp

,Number of Relays
isp,
Digitalocean LLC,714
Contabo Gmbh,408
Contabo Inc.,147
1&1 Ionos Se,102
Contabo Asia Private Limited,53
Akamai Technologies Inc.,44
Akamai International B.V.,41
Comcast,41
Att Services Inc,37
